# Adaptive Tracking Simulation Example

## Setup

In [34]:
from datetime import datetime, timedelta
start_time = datetime.now()

## Radar system

In [35]:
from mpar_sim.radar import PhasedArrayRadar
from mpar_sim.beam.beam import RectangularBeam, GaussianBeam
from mpar_sim.looks.look import Look
from mpar_sim.resource_management import PAPResourceManager
from mpar_sim.schedulers import BestFirstScheduler
import numpy as np

radar = PhasedArrayRadar(
    position=np.array([[0], [0], [0]]),
    position_mapping=(0, 2, 4),
    rotation_offset=np.array([[0], [0], [0]]),
    # Array parameters
    n_elements_x=32,
    n_elements_y=32,
    element_spacing=0.5,  # Wavelengths
    element_tx_power=10,
    # System parameters
    center_frequency=3e9,
    system_temperature=290,
    noise_figure=4,
    # Scan settings
    beam_shape=GaussianBeam,
    az_fov=[-90, 90],
    el_fov=[-90, 90],
    # Detection settings
    false_alarm_rate=1e-6,
)
radar.timestamp = start_time

manager = PAPResourceManager(radar,
                             max_duty_cycle=0.1,
                             max_bandwidth=100e6)
scheduler = BestFirstScheduler(manager,
                               sort_key="start_time",
                               reverse_sort=True,
                               max_time_delta=timedelta(seconds=2))


Raster scan agent

In [36]:
from mpar_sim.agents.raster_scan import RasterScanAgent
import numpy as np

search_agent = RasterScanAgent(
    azimuth_scan_limits=np.array([-30, 30]),
    elevation_scan_limits=np.array([-5, 5]),
    azimuth_beam_spacing=0.8,
    elevation_beam_spacing=0.8,
    azimuth_beamwidth=10,
    elevation_beamwidth=10,
    bandwidth=100e6,
    pulsewidth=1e-6,
    prf=10e3,
    n_pulses=10,
)

## Tracker Components

Create tracker

In [37]:
from stonesoup.measures import Mahalanobis, Euclidean
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.updater.kalman import ExtendedKalmanUpdater
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.gater.distance import DistanceGater

# KF prediction model. Assuming it's matched to the true target model for now
transition_model = CombinedLinearGaussianTransitionModel([
    ConstantVelocity(10),
    ConstantVelocity(10),
    ConstantVelocity(0.0),
])
predictor = KalmanPredictor(transition_model)

updater = ExtendedKalmanUpdater(measurement_model=None)

hypothesizer = DistanceHypothesiser(
    predictor, updater, measure=Mahalanobis(), missed_distance=50)

Create the data associator

In [38]:
from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
associator = GNNWith2DAssignment(hypothesizer)

Create the deleter

In [39]:
from stonesoup.deleter.time import UpdateTimeStepsDeleter, UpdateTimeDeleter
from stonesoup.deleter.error import CovarianceBasedDeleter
from stonesoup.deleter.multi import CompositeDeleter
covar_deleter = CovarianceBasedDeleter(covar_trace_thresh=10)
time_deleter = UpdateTimeStepsDeleter(5)
deleter = CompositeDeleter([covar_deleter, time_deleter], delete_last_pred=True, intersect=True)

Create the initiator

In [40]:
from stonesoup.types.state import GaussianState
from stonesoup.initiator.simple import MultiMeasurementInitiator, SimpleMeasurementInitiator
import numpy as np
from mpar_sim.initiator.initators import MofNInitiator


initiator = MofNInitiator(
    prior_state=GaussianState([0, 0, 0, 0, 0, 0], np.diag([0, 0, 0, 0, 0, 0])),
    measurement_model=None,
    deleter=deleter,
    associator=associator,
    updater=updater,
    confirmation_threshold=[3,5],
)

Tracking agent

In [41]:
from mpar_sim.agents.track_while_scan import TWSAgent
from mpar_sim.agents.adaptive_track import AdaptiveTrackAgent
from mpar_sim.beam.common import aperture2beamwidth

track_agent = AdaptiveTrackAgent(
    initiator,
    associator,
    predictor,
    updater,
    deleter,
    # Adaptive track parameters
    track_sharpness=0.20,
    min_revisit_rate=1,
    max_revisit_rate=5,
    confirm_rate=20,
    # Task parameters
    azimuth_beamwidth=3,
    elevation_beamwidth=3,
    bandwidth=100e6,
    pulsewidth=10e-6,
    prf=5e3,
    n_pulses=100
)

# track_agent = TWSAgent(initiator, associator, updater, deleter)


## Run the simulation

Simulation helper functions

TODO: These would be better served as objects

In [42]:
import copy
import operator
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.array import CovarianceMatrix
from stonesoup.types.state import StateVector
import random
# Set the simulation seed
seed = np.random.randint(0, 2**32-1)
np.random.seed(seed)
random.seed(seed)

# Simulation-level parameters
n_steps = 500
include_noise = True

# Target generation parameters
n_targets_max = 10
initial_state_mean = StateVector([1000, 10, 0, 0, 0, 0])
initial_state_covariance = CovarianceMatrix(np.diag([200, 50, 200, 50, 0, 0]))
initial_state = GaussianState(initial_state_mean, initial_state_covariance)
death_probability = 0.0
birth_probability = 0.1
target_rcs = 1


truths = []
all_truths = []
confirmed_tracks = set()
tentative_tracks = set()
all_measurements = []
all_tracks = set()

# Simulation time variables.
time = start_time
scheduler_time = start_time
for istep in range(n_steps):
  ########################################
  # Allocate resources 
  ########################################
  # Generate looks from each agent
  search_look = search_agent.act(current_time=scheduler_time)
  track_looks = track_agent.act(current_time=scheduler_time)
  looks = [search_look] + track_looks

  # Schedule new looks, sorted so that the task with the nearest end time is selected first below
  scheduler.schedule(looks, scheduler_time)
  manager.allocated_tasks.sort(
      key=operator.attrgetter("end_time"), reverse=True)
  
  ########################################
  # Update scenario
  ########################################
  # Minimum start time of all scheduled tasks. When this changes, a new "batch" of tasks has been allocated and the scenario needs to be updated
  min_start_time = min([task.start_time for task in manager.allocated_tasks]
                       ) if manager.allocated_tasks else time
  # Update targets
  # This should only happen when the "main" simulation time is stepped forward
  if min_start_time > time:
    dt = min_start_time - time
    time = min_start_time
    # Delete targets according to the death process
    truths = [truth for truth in truths if np.random.rand() > death_probability]

    # Targets, be reborn!
    for _ in range(np.random.poisson(birth_probability)):
      # Sample an initial state from the mean and covariance defined above
      state_vector = initial_state.state_vector + \
          initial_state.covar @ np.random.randn(initial_state.ndim, 1)
      if state_vector[0] < 0:
        state_vector[0] = -state_vector[0]
      if state_vector[1] < 0:
        state_vector[1] = -state_vector[1]
      state = GroundTruthState(
          state_vector=state_vector,
          timestamp=time,
      )
      # Give the target an RCS
      # TODO: Create a GroundTruthTarget class with an RCS attribute
      state.rcs = target_rcs
      # Add to the list of truths
      truth = GroundTruthPath([state])
      truths.append(truth)
      all_truths.append(truth)

    # Also randomly delete targets if we have exceeded the maximum target count
    if len(truths) > n_targets_max:
      indices = np.random.choice(
          len(truths), len(truths) - n_targets_max, replace=False)
      for index in sorted(indices, reverse=True):
        del truths[index]
        
    # Update targets
    for truth in truths:
      state = np.asarray(truth[-1].state_vector)
      truth.append(GroundTruthState(
          transition_model.function(truth[-1],
                                    noise=include_noise,
                                    time_interval=dt),
          timestamp=time))
      truth[-1].rcs = target_rcs

  ########################################
  # Simulate the current look
  ########################################
  look = manager.allocated_tasks.pop()
  scheduler_time = look.end_time
  radar.load_look(look)
  if look.start_time > time:
    # Interpolate targets to the start of the look
    current_truths = []
    dt = look.start_time - time
    for truth in current_truths:
      if radar.is_detectable(truth[-1]):
        state = GroundTruthState(
            transition_model.function(truth[-1],
                                      noise=False,
                                      time_interval=dt),
            timestamp=look.start_time)
        state.rcs = target_rcs
        truth = GroundTruthPath([state])
        current_truths.append(truth)
  else:
    current_truths = truths

  detections = radar.measure(current_truths, noise=include_noise)

  # Update tracks
  confirmed_tracks = track_agent.update_tracks(detections, look.start_time)

  # Update metrics
  all_measurements.append(detections)
  all_tracks |= confirmed_tracks


In [43]:
print(len(confirmed_tracks))
print(len(detections))
print(len(truths))

1
0
10


## Plot simulation results

In [ ]:
from stonesoup.plotter import Plotterly

plotter = Plotterly()
plotter.plot_sensors(radar, "Radar")
plotter.plot_ground_truths(all_truths, [0, 2])
plotter.plot_measurements(all_measurements, [0, 2])
plotter.plot_tracks(all_tracks, [0,2])
plotter.fig